In [32]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', 350)

In [33]:
import os
def get_project_root_path():
    project_directory = "fifa22-players-analysis"
    separated_path = os.getcwd().split(os.sep)
    project_directory_index = separated_path.index(project_directory) + 1
    return os.sep.join(separated_path[0:project_directory_index])


import joblib
def save_model(model, model_name):
    filename = f"model_name_{model_name}_{str(date.today())}.pkl"
    print(filename)
    model_path = os.sep.join([get_project_root_path(), "saved_models", filename])
    joblib.dump(model, model_path)

In [34]:
data = pd.read_csv(f"data/processed_data_{date.today()}.tsv", sep="\t")
data.head()

/tmp/ipykernel_108936/2624993333.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f"data/processed_data_{date.today()}.tsv", sep="\t")


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,bmi,obese,underweight,trait Cautious With Crosses,trait Chip Shot (AI),trait Comes For Crosses,trait Dives Into Tackles (AI),trait Early Crosser,trait Finesse Shot,trait Flair,trait GK Long Throw,trait Giant Throw-in,trait Injury Prone,trait Leadership,trait Long Passer (AI),trait Long Shot Taker (AI),trait Long Throw-in,trait One Club Player,trait Outside Foot Shot,trait Playmaker (AI),trait Power Free-Kick,trait Power Header,trait Rushes Out Of Goal,trait Saves with Feet,trait Solid Player,trait Speed Dribbler (AI),trait Team Player,trait Technical Dribbler (AI),tag #Acrobat,tag #Aerial Threat,tag #Clinical Finisher,tag #Complete Defender,tag #Complete Forward,tag #Complete Midfielder,tag #Crosser,tag #Distance Shooter,tag #Dribbler,tag #Engine,tag #FK Specialist,tag #Playmaker,tag #Poacher,tag #Speedster,tag #Strength,tag #Tackling,tag #Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,work rate High/High,work rate High/Low,work rate High/Medium,work rate Low/High,work rate Low/Low,work rate Low/Medium,work rate Medium/High,work rate Medium/Low,work rate Medium/Medium,body type Lean (170-),body type Lean (170-185),body type Lean (185+),body type Normal (170-),body type Normal (170-185),body type Normal (185+),body type Stocky (170-),body type Stocky (170-185),body type Stocky (185+),body type Unique,preferred foot Left,preferred foot Right,nationality Afghanistan,nationality Albania,nationality Algeria,nationality Andorra,nationality Angola,nationality Antigua and Barbuda,nationality Argentina,nationality Armenia,nationality Australia,nationality Austria,nationality Azerbaijan,nationality Barbados,nationality Belarus,nationality Belgium,nationality Belize,nationality Benin,nationality Bermuda,nationality Bhutan,nationality Bolivia,nationality Bosnia and Herzegovina,nationality Brazil,nationality Bulgaria,nationality Burkina Faso,nationality Burundi,nationality Cameroon,nationality Canada,nationality Cape Verde Islands,nationality Central African Republic,nationality Chad,nationality Chile,nationality China PR,nationality Chinese Taipei,nationality Colombia,nationality Comoros,nationality Congo,nationality Congo DR,nationality Costa Rica,nationality Croatia,nationality Cuba,nationality Curacao,nationality Cyprus,nationality Czech Republic,nationality Côte d'Ivoire,nationality Denmark,nationality Dominican Republic,nationality Ecuador,nationality Egypt,nationality El Salvador,nationality England,nationality Equatorial Guinea,nationality Eritrea,nationality Estonia,nationality Ethiopia,nationality Faroe Islands,nationality Fiji,nationality Finland,nationality France,nationality Gabon,nationality Gambia,nationality Georgia,nationality Germany,nationality Ghana,nationality Gibraltar,nationality Greece,nationality Grenada,nationality Guam,nationality Guatemala,nationality Guinea,nationality Guinea Bissau,nationality Guyana,nationality 

In [35]:
def count_na(df: pd.DataFrame) -> None:
    nan_sum = df.isna().sum()
    display(nan_sum[nan_sum > 0])

count_na(data)

value_eur                74
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
player_tags           17798
player_traits          9841
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

Analisando essa tabela com os valores nulos podemos perceber que alguns jogadores do dataset não possuem um time atual, isso explica
`wage_eur`, `club_team_id`, `club_position`, `club_jersey_number` terem a mesma quantidade de valores nulos (61),
porém `value_eur` deveria seguir esse mesmo padrão, assim podemos dizer que 13 (74 - 61) jogadores que estão ativos não possuem um valor,
portanto precisamos prever esses dados.

Também podemos perceber que temos 2132 jogadores não possuem `pace`, `shooting`, `passing`, `dribbling`, `defending`, `physic`. Vamos analisar esses jogadores

In [36]:
def percent_of_gk(df: pd.DataFrame) -> None:
    gk = df[df['GK'] == 1]
    display(gk.shape[0] / df.shape[0])


missing_main_attr_data = data[data.pace.isna()]

percent_of_gk(missing_main_attr_data)
missing_main_attr_data

1.0

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,bmi,obese,underweight,trait Cautious With Crosses,trait Chip Shot (AI),trait Comes For Crosses,trait Dives Into Tackles (AI),trait Early Crosser,trait Finesse Shot,trait Flair,trait GK Long Throw,trait Giant Throw-in,trait Injury Prone,trait Leadership,trait Long Passer (AI),trait Long Shot Taker (AI),trait Long Throw-in,trait One Club Player,trait Outside Foot Shot,trait Playmaker (AI),trait Power Free-Kick,trait Power Header,trait Rushes Out Of Goal,trait Saves with Feet,trait Solid Player,trait Speed Dribbler (AI),trait Team Player,trait Technical Dribbler (AI),tag #Acrobat,tag #Aerial Threat,tag #Clinical Finisher,tag #Complete Defender,tag #Complete Forward,tag #Complete Midfielder,tag #Crosser,tag #Distance Shooter,tag #Dribbler,tag #Engine,tag #FK Specialist,tag #Playmaker,tag #Poacher,tag #Speedster,tag #Strength,tag #Tackling,tag #Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,work rate High/High,work rate High/Low,work rate High/Medium,work rate Low/High,work rate Low/Low,work rate Low/Medium,work rate Medium/High,work rate Medium/Low,work rate Medium/Medium,body type Lean (170-),body type Lean (170-185),body type Lean (185+),body type Normal (170-),body type Normal (170-185),body type Normal (185+),body type Stocky (170-),body type Stocky (170-185),body type Stocky (185+),body type Unique,preferred foot Left,preferred foot Right,nationality Afghanistan,nationality Albania,nationality Algeria,nationality Andorra,nationality Angola,nationality Antigua and Barbuda,nationality Argentina,nationality Armenia,nationality Australia,nationality Austria,nationality Azerbaijan,nationality Barbados,nationality Belarus,nationality Belgium,nationality Belize,nationality Benin,nationality Bermuda,nationality Bhutan,nationality Bolivia,nationality Bosnia and Herzegovina,nationality Brazil,nationality Bulgaria,nationality Burkina Faso,nationality Burundi,nationality Cameroon,nationality Canada,nationality Cape Verde Islands,nationality Central African Republic,nationality Chad,nationality Chile,nationality China PR,nationality Chinese Taipei,nationality Colombia,nationality Comoros,nationality Congo,nationality Congo DR,nationality Costa Rica,nationality Croatia,nationality Cuba,nationality Curacao,nationality Cyprus,nationality Czech Republic,nationality Côte d'Ivoire,nationality Denmark,nationality Dominican Republic,nationality Ecuador,nationality Egypt,nationality El Salvador,nationality England,nationality Equatorial Guinea,nationality Eritrea,nationality Estonia,nationality Ethiopia,nationality Faroe Islands,nationality Fiji,nationality Finland,nationality France,nationality Gabon,nationality Gambia,nationality Georgia,nationality Germany,nationality Ghana,nationality Gibraltar,nationality Greece,nationality Grenada,nationality Guam,nationality Guatemala,nationality Guinea,nationality Guinea Bissau,nationality Guyana,nationality 

Como vimos que os goleiros não tem os atributos listados acima, mas possuem um overall em cada posição, seria interessante prever esses atributos.

In [37]:
missing_gk_speed_data = data[data.goalkeeping_speed.isna()]
sum_of_missing_data = missing_gk_speed_data.shape[0] + missing_main_attr_data.shape[0]
print(f"Jogadores sem gk speed + jogadores sem atributos primários: {sum_of_missing_data}")
print(f"Numero de jogadores: {data.shape[0]}")

Jogadores sem gk speed + jogadores sem atributos primários: 19239
Numero de jogadores: 19239


Assim podemos ver que todos os jogadores que não são goleiros não possuem `goalkeeping_speed`

## Prevendo valor dos jogadores
Nessa etapa vamos criar um modelo para prever o valor dos 13 jogadores que não possuem valor

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

def predict_and_populate(df_to_populate, df_to_train, model, column_to_predict, round_=None, save=False) -> pd.DataFrame:
    # Separating the target variable
    target = df_to_train[column_to_predict]
    features = df_to_train.drop(column_to_predict, axis=1)

    # Poluting the missing features
    mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    mean_imputer.fit(features)
    imputed_features = mean_imputer.transform(features)

    # Train test split
    features_train, features_test, target_train, target_test = train_test_split(imputed_features, target, test_size=0.2)

    # Training the model
    trained_model = model.fit(features_train, target_train)

    # Saving the model
    if save:
        model_name = str(model).split("(")[0]
        filename = f"{model_name}_{column_to_predict}"
        save_model(trained_model, filename)

    # Calculatin model score
    model_score = trained_model.score(features_test, target_test)
    print(f"Score do modelo: {model_score:.4f}")

    # Populating the missing values
    feature_to_populate = mean_imputer.transform(df_to_populate.drop(column_to_predict, axis=1))
    predicted_values = trained_model.predict(feature_to_populate)

    if round_ is not None:
        predicted_values = predicted_values.round(round_)

    df_to_populate[column_to_predict] = predicted_values

    return df_to_populate

In [39]:
players_with_club_and_no_value = data.query("not club_team_id.isna() and value_eur.isna()").copy()
data_without_players_with_club_and_no_value = data.drop(players_with_club_and_no_value.index)

data_to_train = data_without_players_with_club_and_no_value.query("not value_eur.isna()").copy()
columns_to_drop = [
    "sofifa_id", "player_url", "short_name", "long_name",
    "dob", "club_position","player_traits", "player_tags",
    "player_positions", "work_rate", "body_type",
    "preferred_foot", "nationality_name"
]

data_to_train = data_to_train.drop(columns=columns_to_drop)
data_to_fill = players_with_club_and_no_value.drop(columns=columns_to_drop)

In [40]:
# Normalizando os dados
# from sklearn.preprocessing import StandardScaler
#
# standerd_scaler =  StandardScaler()
# standardized_features = standerd_scaler.fit_transform(imputed_features)
# standardized_features

## Comparando modelos diferentes

In [41]:
# TODO ver tanto o MSE quanto o MAE
# TODO aplicar RFECV em cada modelo

from sklearn.dummy import DummyRegressor

dummy = DummyRegressor()
predict_and_populate(data_to_fill, data_to_train, dummy, "value_eur");

Score do modelo: -0.0003


In [42]:
from sklearn.linear_model import LinearRegression

linear_regression = LinearRegression()
predict_and_populate(data_to_fill, data_to_train, linear_regression, "value_eur");

Score do modelo: 0.9733


In [43]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(random_state=0, n_jobs=-1, oob_score=True)
players_with_club_and_no_value = predict_and_populate(
    data_to_fill, data_to_train, random_forest, "value_eur", save=True
);

model_name_RandomForestRegressor_value_eur_2022-06-21.pkl
Score do modelo: 0.9810


### Preenchendo os dados a partir do modelo criado

In [44]:
data.loc[players_with_club_and_no_value.index, "value_eur"] = players_with_club_and_no_value.value_eur
data.loc[players_with_club_and_no_value.index]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,bmi,obese,underweight,trait Cautious With Crosses,trait Chip Shot (AI),trait Comes For Crosses,trait Dives Into Tackles (AI),trait Early Crosser,trait Finesse Shot,trait Flair,trait GK Long Throw,trait Giant Throw-in,trait Injury Prone,trait Leadership,trait Long Passer (AI),trait Long Shot Taker (AI),trait Long Throw-in,trait One Club Player,trait Outside Foot Shot,trait Playmaker (AI),trait Power Free-Kick,trait Power Header,trait Rushes Out Of Goal,trait Saves with Feet,trait Solid Player,trait Speed Dribbler (AI),trait Team Player,trait Technical Dribbler (AI),tag #Acrobat,tag #Aerial Threat,tag #Clinical Finisher,tag #Complete Defender,tag #Complete Forward,tag #Complete Midfielder,tag #Crosser,tag #Distance Shooter,tag #Dribbler,tag #Engine,tag #FK Specialist,tag #Playmaker,tag #Poacher,tag #Speedster,tag #Strength,tag #Tackling,tag #Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,work rate High/High,work rate High/Low,work rate High/Medium,work rate Low/High,work rate Low/Low,work rate Low/Medium,work rate Medium/High,work rate Medium/Low,work rate Medium/Medium,body type Lean (170-),body type Lean (170-185),body type Lean (185+),body type Normal (170-),body type Normal (170-185),body type Normal (185+),body type Stocky (170-),body type Stocky (170-185),body type Stocky (185+),body type Unique,preferred foot Left,preferred foot Right,nationality Afghanistan,nationality Albania,nationality Algeria,nationality Andorra,nationality Angola,nationality Antigua and Barbuda,nationality Argentina,nationality Armenia,nationality Australia,nationality Austria,nationality Azerbaijan,nationality Barbados,nationality Belarus,nationality Belgium,nationality Belize,nationality Benin,nationality Bermuda,nationality Bhutan,nationality Bolivia,nationality Bosnia and Herzegovina,nationality Brazil,nationality Bulgaria,nationality Burkina Faso,nationality Burundi,nationality Cameroon,nationality Canada,nationality Cape Verde Islands,nationality Central African Republic,nationality Chad,nationality Chile,nationality China PR,nationality Chinese Taipei,nationality Colombia,nationality Comoros,nationality Congo,nationality Congo DR,nationality Costa Rica,nationality Croatia,nationality Cuba,nationality Curacao,nationality Cyprus,nationality Czech Republic,nationality Côte d'Ivoire,nationality Denmark,nationality Dominican Republic,nationality Ecuador,nationality Egypt,nationality El Salvador,nationality England,nationality Equatorial Guinea,nationality Eritrea,nationality Estonia,nationality Ethiopia,nationality Faroe Islands,nationality Fiji,nationality Finland,nationality France,nationality Gabon,nationality Gambia,nationality Georgia,nationality Germany,nationality Ghana,nationality Gibraltar,nationality Greece,nationality Grenada,nationality Guam,nationality Guatemala,nationality Guinea,nationality Guinea Bissau,nationality Guyana,nationality 

In [45]:
count_na(data)

value_eur                61
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
player_tags           17798
player_traits          9841
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

In [46]:
# Removendo jogadores não ativos
data = data.query("not club_team_id.isna()").copy()

count_na(data)

release_clause_eur     1115
player_tags           17744
player_traits          9823
pace                   2124
shooting               2124
passing                2124
dribbling              2124
defending              2124
physic                 2124
goalkeeping_speed     17054
dtype: int64

# Prevendo pace, shooting, passing, dribbling, defending, physic dos goleiros

In [47]:
gk_data = data.query("GK == 1").copy()
data_without_gk = data.query("GK == 0").copy()

columns_to_predict = ["pace", "shooting", "passing", "dribbling", "defending", "physic"]

for column_to_predict in columns_to_predict:

    columns_to_drop_with_other_atributes = [col for col in columns_to_predict if col != column_to_predict]
    columns_to_drop_with_other_atributes.extend(columns_to_drop)

    data_to_train = data_without_gk.drop(columns=columns_to_drop_with_other_atributes)
    data_to_predict = gk_data.drop(columns=columns_to_drop_with_other_atributes)

    linear_regression = LinearRegression()
    print(f"Prevendo {column_to_predict}")
    filled_data = predict_and_populate(
        data_to_predict, data_to_train, linear_regression,
        column_to_predict, 0, save=True
    )
    print()

    data.loc[filled_data.index, column_to_predict] = filled_data[column_to_predict]

# data.loc[gk_data.index]

Prevendo pace
model_name_LinearRegression_pace_2022-06-21.pkl
Score do modelo: 0.9993

Prevendo shooting
model_name_LinearRegression_shooting_2022-06-21.pkl
Score do modelo: 0.9991

Prevendo passing
model_name_LinearRegression_passing_2022-06-21.pkl
Score do modelo: 0.9961

Prevendo dribbling
model_name_LinearRegression_dribbling_2022-06-21.pkl
Score do modelo: 0.9991

Prevendo defending
model_name_LinearRegression_defending_2022-06-21.pkl
Score do modelo: 0.9997

Prevendo physic
model_name_LinearRegression_physic_2022-06-21.pkl
Score do modelo: 0.9992



In [48]:
count_na(data)

release_clause_eur     1115
player_tags           17744
player_traits          9823
goalkeeping_speed     17054
dtype: int64

## Prevendo goalkeeping_speed dos jogadores

In [49]:
players_data = data.query("GK == 0").copy()
players_data

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,nationality_name,age,bmi,obese,underweight,trait Cautious With Crosses,trait Chip Shot (AI),trait Comes For Crosses,trait Dives Into Tackles (AI),trait Early Crosser,trait Finesse Shot,trait Flair,trait GK Long Throw,trait Giant Throw-in,trait Injury Prone,trait Leadership,trait Long Passer (AI),trait Long Shot Taker (AI),trait Long Throw-in,trait One Club Player,trait Outside Foot Shot,trait Playmaker (AI),trait Power Free-Kick,trait Power Header,trait Rushes Out Of Goal,trait Saves with Feet,trait Solid Player,trait Speed Dribbler (AI),trait Team Player,trait Technical Dribbler (AI),tag #Acrobat,tag #Aerial Threat,tag #Clinical Finisher,tag #Complete Defender,tag #Complete Forward,tag #Complete Midfielder,tag #Crosser,tag #Distance Shooter,tag #Dribbler,tag #Engine,tag #FK Specialist,tag #Playmaker,tag #Poacher,tag #Speedster,tag #Strength,tag #Tackling,tag #Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST,work rate High/High,work rate High/Low,work rate High/Medium,work rate Low/High,work rate Low/Low,work rate Low/Medium,work rate Medium/High,work rate Medium/Low,work rate Medium/Medium,body type Lean (170-),body type Lean (170-185),body type Lean (185+),body type Normal (170-),body type Normal (170-185),body type Normal (185+),body type Stocky (170-),body type Stocky (170-185),body type Stocky (185+),body type Unique,preferred foot Left,preferred foot Right,nationality Afghanistan,nationality Albania,nationality Algeria,nationality Andorra,nationality Angola,nationality Antigua and Barbuda,nationality Argentina,nationality Armenia,nationality Australia,nationality Austria,nationality Azerbaijan,nationality Barbados,nationality Belarus,nationality Belgium,nationality Belize,nationality Benin,nationality Bermuda,nationality Bhutan,nationality Bolivia,nationality Bosnia and Herzegovina,nationality Brazil,nationality Bulgaria,nationality Burkina Faso,nationality Burundi,nationality Cameroon,nationality Canada,nationality Cape Verde Islands,nationality Central African Republic,nationality Chad,nationality Chile,nationality China PR,nationality Chinese Taipei,nationality Colombia,nationality Comoros,nationality Congo,nationality Congo DR,nationality Costa Rica,nationality Croatia,nationality Cuba,nationality Curacao,nationality Cyprus,nationality Czech Republic,nationality Côte d'Ivoire,nationality Denmark,nationality Dominican Republic,nationality Ecuador,nationality Egypt,nationality El Salvador,nationality England,nationality Equatorial Guinea,nationality Eritrea,nationality Estonia,nationality Ethiopia,nationality Faroe Islands,nationality Fiji,nationality Finland,nationality France,nationality Gabon,nationality Gambia,nationality Georgia,nationality Germany,nationality Ghana,nationality Gibraltar,nationality Greece,nationality Grenada,nationality Guam,nationality Guatemala,nationality Guinea,nationality Guinea Bissau,nationality Guyana,nationality 

In [50]:
data_to_predict = players_data.drop(columns=columns_to_drop)
data_to_train = gk_data.drop(columns=columns_to_drop)

column_to_predict = "goalkeeping_speed"

linear_regression = LinearRegression()
filled_data = predict_and_populate(
    data_to_predict, data_to_train, linear_regression, column_to_predict, 0, save=True
)

data.loc[filled_data.index, column_to_predict] = filled_data[column_to_predict]

model_name_LinearRegression_goalkeeping_speed_2022-06-21.pkl
Score do modelo: 0.9992


In [51]:
count_na(data)

release_clause_eur     1115
player_tags           17744
player_traits          9823
dtype: int64

In [52]:
from datetime import date
data.drop(columns=["release_clause_eur"], inplace=True)

filled_data_save_path = os.sep.join([get_project_root_path(), "preprocessing", "data", f"filled_data_{date.today()}.tsv"])
data.to_csv(filled_data_save_path, index=False, sep="\t")
filled_data_save_path

'/home/gustavo/fifa22-players-analysis/preprocessing/data/filled_data_2022-06-21.tsv'

In [53]:
count_na(data)

player_tags      17744
player_traits     9823
dtype: int64

In [54]:
data.describe().T.round()

,count,mean,std,min,25%,50%,75%,max
sofifa_id,19178.0,231514.0,27028.0,41.0,214568.0,236576.0,253563.0,264640.0
overall,19178.0,66.0,7.0,47.0,61.0,66.0,70.0,93.0
potential,19178.0,71.0,6.0,49.0,67.0,71.0,75.0,95.0
value_eur,19178.0,2849353.0,7611306.0,9000.0,475000.0,975000.0,2000000.0,194000000.0
wage_eur,19178.0,9018.0,19470.0,500.0,1000.0,3000.0,8000.0,350000.0
...,...,...,...,...,...,...,...,...
nationality Venezuela,19178.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
nationality Vietnam,19178.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
nationality Wales,19178.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
nationality Zambia,19178.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
